In [1]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [5]:
spark = SparkSession.builder.master("local[4]").appName("1280").getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/23 01:17:17 WARN Utils: Your hostname, de24, resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface enp0s3)
25/07/23 01:17:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/23 01:17:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
'''
Table: Students

+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| student_id    | int     |
| student_name  | varchar |
+---------------+---------+
student_id is the primary key (column with unique values) for this table.
Each row of this table contains the ID and the name of one student in the school.
 

Table: Subjects

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| subject_name | varchar |
+--------------+---------+
subject_name is the primary key (column with unique values) for this table.
Each row of this table contains the name of one subject in the school.
 

Table: Examinations

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| student_id   | int     |
| subject_name | varchar |
+--------------+---------+
There is no primary key (column with unique values) for this table. It may contain duplicates.
Each student from the Students table takes every course from the Subjects table.
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.
 

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

 

Example 1:

Input: 
Students table:
+------------+--------------+
| student_id | student_name |
+------------+--------------+
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |
+------------+--------------+
Subjects table:
+--------------+
| subject_name |
+--------------+
| Math         |
| Physics      |
| Programming  |
+--------------+
Examinations table:
+------------+--------------+
| student_id | subject_name |
+------------+--------------+
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |
+------------+--------------+
Output: 
+------------+--------------+--------------+----------------+
| student_id | student_name | subject_name | attended_exams |
+------------+--------------+--------------+----------------+
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |
+------------+--------------+--------------+----------------+
Explanation: 
The result table should contain all students and all subjects.
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.
Alex did not attend any exams.
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.
'''

In [16]:
data_students = [
(1 ,'Alice'),
(2 ,'Bob'  ),
(13,'John' ),
(6 ,'Alex' )
]
students = ['student_id','student_name']

## Melwin Please Note when ever there is a single data there must be a taling comma 
data_subject = [
('Math',),
('Physics',),
('Programming',)
]

subject = ['subject_name']

data_examinations = [
(1 ,'Math'       ),
(1 ,'Physics'    ),
(1 ,'Programming'),
(2 ,'Programming'),
(1 ,'Physics'    ),
(1 ,'Math'       ),
(13,'Math'       ),
(13,'Programming'),
(13,'Physics'    ),
(2 ,'Math'       ),
(1 ,'Math'       ),
]

examinations = ['student_id','subject_name']

In [17]:
df_students = spark.createDataFrame(data =data_students ,schema = students)
df_subject = spark.createDataFrame(data =data_subject ,schema = subject)
df_examinations = spark.createDataFrame(data =data_examinations ,schema = examinations)

In [18]:
spark.conf.set("spark.sql.crossJoin.enabled", True)

In [30]:
df_students.alias("st").crossJoin(df_subject.alias("su"))\
                       .join(df_examinations.alias("e"), (F.col("st.student_id") == F.col("e.student_id")) &
                                                         (F.col("su.subject_name") == F.col("e.subject_name"))
                             , "left")\
                       .groupBy(F.col("st.student_id"),F.col("st.student_name"),F.col("su.subject_name"))\
                       .agg(F.count(F.col("e.student_id")).alias("attended_exams"))\
                       .orderBy(F.col("st.student_id"),F.col("su.subject_name"))\
                       .show()

+----------+------------+------------+--------------+
|student_id|student_name|subject_name|attended_exams|
+----------+------------+------------+--------------+
|         1|       Alice|        Math|             3|
|         1|       Alice|     Physics|             2|
|         1|       Alice| Programming|             1|
|         2|         Bob|        Math|             1|
|         2|         Bob|     Physics|             0|
|         2|         Bob| Programming|             1|
|         6|        Alex|        Math|             0|
|         6|        Alex|     Physics|             0|
|         6|        Alex| Programming|             0|
|        13|        John|        Math|             1|
|        13|        John|     Physics|             1|
|        13|        John| Programming|             1|
+----------+------------+------------+--------------+



## SQL Implementation

<pre>
SELECT Students.student_id, Students.student_name, Subjects.subject_name, COUNT(Examinations.student_id)
FROM easy_1280_Students Students 
JOIN easy_1280_Subjects Subjects on 1=1
LEFT JOIN easy_1280_Examinations Examinations ON 
          Students.student_id = Examinations.student_id 
      AND Subjects.subject_name = Examinations.subject_name
GROUP BY Students.student_id, Students.student_name, Subjects.subject_name
ORDER BY Students.student_id, Subjects.subject_name;
</pre>